In [76]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
import json
import datetime


from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn.cluster import AgglomerativeClustering
#import scipy.cluster.hierarchy as sch

#import April12.ipynb
#%store -r icd_grouped 
import statsmodels.api as sm
from patsy import dmatrices 

# model selection
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

import seaborn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

In [77]:
from transforms.transform_function import TransformFunction
from scipy.stats import norm, truncnorm
import numpy as np
from concurrent.futures import ProcessPoolExecutor

from expectation_maximization import ExpectationMaximization
from batch_expectation_maximization import BatchExpectationMaximization

#Users.kathy908000.github.online_mixed_gc_imp.em.batch_expectation_maximization.py import BatchExpectationMaximization



In [78]:
df= pd.read_csv("/Users/kathy908000/github/health-care-research/data/Diabetes_Numerical.csv")
df.head(30)

,SUBJECT_ID,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,...,NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER,HISPANIC/LATINO - CENTRAL AMERICAN (OTHER),ASIAN - JAPANESE,ASIAN - THAI,HISPANIC/LATINO - HONDURAN,HISPANIC/LATINO - CUBAN,MIDDLE EASTERN,ASIAN - OTHER,HISPANIC/LATINO - MEXICAN,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE
0,13,3.6660,1,4,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,18,1.2885,1,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20,1.0508,1,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21,14.2664,2,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,24,0.5124,1,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,25,3.5466,1,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,28,1.1224,1,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,35,5.3757,1,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,37,1.1397,1,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,55,1.9138,1,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
df.columns.tolist()
print(df.columns.tolist())

['SUBJECT_ID', 'LOS', 'HOSPITALIZATION', 'ORDINAL_AGE', 'ELECTIVE', 'EMERGENCY', 'NEWBORN', 'URGENT', 'Government', 'Medicaid', 'Medicare', 'Private', 'Self Pay', 'DIVORCED', 'LIFE PARTNER', 'MARRIED', 'NAN', 'SEPARATED', 'SINGLE', 'UNKNOWN', 'WIDOWED', 'GENDER', '3', '4', '5', '7', '8', '9', '10', '11', '12', '13', '14', '18', '27', '31', '32', '34', '35', '38', '39', '40', '41', '42', '46', '47', '48', '49', '52', '53', '54', '57', '58', '66', '70', '75', '78', '79', '82', '86', '88', '94', '96', '97', '110', '111', '112', '117', '120', '121', '127', '130', '131', '135', '136', '137', '138', '139', '140', '141', '142', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '161', '162', '163', '164', '171', '172', '173', '174', '175', '176', '180', '182', '183', '184', '185', '186', '187', '188', '189', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '207', '208', '209', 

In [80]:
sample100= df.sample(n=100)
sample100.head()

,SUBJECT_ID,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,...,NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER,HISPANIC/LATINO - CENTRAL AMERICAN (OTHER),ASIAN - JAPANESE,ASIAN - THAI,HISPANIC/LATINO - HONDURAN,HISPANIC/LATINO - CUBAN,MIDDLE EASTERN,ASIAN - OTHER,HISPANIC/LATINO - MEXICAN,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE
3691,20643,67.1683,24,7,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5193,28485,3.1076,1,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5999,32555,0.9631,1,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9593,89797,31.5519,3,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6493,46123,5.8186,1,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
sample100.shape

(100, 921)

In [82]:
sample100
c = [0,1,2,3]
samplebi= sample100.drop(df.columns[c],axis=1)

samplebi.head()

,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,DIVORCED,...,NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER,HISPANIC/LATINO - CENTRAL AMERICAN (OTHER),ASIAN - JAPANESE,ASIAN - THAI,HISPANIC/LATINO - HONDURAN,HISPANIC/LATINO - CUBAN,MIDDLE EASTERN,ASIAN - OTHER,HISPANIC/LATINO - MEXICAN,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE
3691,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5193,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5999,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9593,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6493,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
#X = sample100
X = samplebi

# Batch Expectation Maximization

In [64]:
BEM = BatchExpectationMaximization()

In [65]:
im = BEM.impute_missing(X, cont_indices=None, ord_indices=None, threshold=0.01, max_iter=100, max_workers=None, max_ord=100, batch_size=64, num_ord_updates=2)
#im = BEM.impute_missing(X, cont_indices=[0,1,2], ord_indices=[3], threshold=0.01, max_iter=100, max_workers=None, max_ord=100, batch_size=64, num_ord_updates=2)


TypeError: 'int' object is not subscriptable

# Very small matrix

In [67]:
small = {'A':[0, 1, 0, 0], 'Age':[20, 21, 19, 18]} 
small = pd.DataFrame(small) 
small

,A,Age
0,0,20
1,1,21
2,0,19
3,0,18


In [68]:
X =small

In [74]:
im = BEM.impute_missing(X, cont_indices=[0,1], ord_indices=None, threshold=0.01, max_iter=100, max_workers=None, max_ord=100, batch_size=64, num_ord_updates=2)





TypeError: '(slice(None, None, None), [0, 1])' is an invalid key

In [73]:
covar = BEM._fit_covariance(X, cont_indices, ord_indices, threshold, max_iter, max_workers, batch_size, num_ord_updates)

NameError: name 'cont_indices' is not defined

# expectation_maximization

In [97]:
X = small
#X = small.values

In [98]:
EM = ExpectationMaximization()

In [99]:
emim = EM.impute_missing(X, cont_indices=[0,1], ord_indices=None, threshold=0.01, max_iter=100, max_workers=None, max_ord=100)


TypeError: '(slice(None, None, None), [0, 1])' is an invalid key

In [102]:
emim = EM.impute_missing(X, cont_indices=None, ord_indices=None, threshold=0.01, max_iter=100, max_workers=None, max_ord=100)

TypeError: 'int' object is not subscriptable

In [100]:
emfit = EM._fit_covariance(X, cont_indices=[0], ord_indices=[1], threshold=2, max_iter=10, max_workers=None)



TypeError: '(slice(None, None, None), None)' is an invalid key